In [41]:
import numpy as np
import pandas as pd
import optuna
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.manifold import TSNE
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import optim
#from geomloss import SamplesLoss
from torch.autograd import Function
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.nn.functional import normalize
#from torchmetrics.classification import BinaryAccuracy
#from torchmetrics.classification import BinaryF1Score
torch.manual_seed(0)

In [42]:
class TarNet(nn.Module):
    def __init__(self,params):
        super(TarNet, self).__init__()
        self.encoder1 = nn.Linear(25, params['RL11'])
        self.encoder2 = nn.Linear(params['RL11'], params['RL21'])
        self.encoder3 = nn.Linear(params['RL21'], params['RL32'])

        self.regressor1_y0 = nn.Sequential(
            nn.Linear(params['RL32'], params['RG012']),
            nn.ELU(),
            nn.Dropout(p=.01),
        )
        self.regressor2_y0 = nn.Sequential(
            nn.Linear(params['RG012'], params['RG022']),
            nn.ELU(),
            nn.Dropout(p=.01),
        )
        self.regressorO_y0 = nn.Linear(params['RG022'], 1)

        self.regressor1_y1 = nn.Sequential(
            nn.Linear(params['RL32'], params['RG112']),
            nn.ELU(),
            nn.Dropout(p=.01),
        )
        self.regressor2_y1 = nn.Sequential(
            nn.Linear(params['RG112'], params['RG122']),
            nn.ELU(),
            nn.Dropout(p=.01),
        )
        self.regressorO_y1 = nn.Linear(params['RG122'], 1)


    def forward(self, inputs):
        x = nn.functional.elu(self.encoder1(inputs))
        x = nn.functional.elu(self.encoder2(x))
        phi = nn.functional.elu(self.encoder3(x))

        out_y0 = self.regressor1_y0(phi)
        out_y0 = self.regressor2_y0(out_y0)
        y0 = self.regressorO_y0(out_y0)

        out_y1 = self.regressor1_y1(phi)
        out_y1 = self.regressor2_y1(out_y1)
        y1 = self.regressorO_y1(out_y1)

        concat = torch.cat((y0, y1), 1)
        return concat

In [43]:
def objective(trial,i):

    params = {
          'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-3),
          'optimizer': trial.suggest_categorical("optimizer", ["Adam", "SGD"]),
          'batch_size':trial.suggest_int('batch_size', 8, 256),
          'RL11':trial.suggest_int('RL11', 16, 512),
          'RL21': trial.suggest_int('RL21', 16, 512),
          'RL32': trial.suggest_int('RL32', 16, 512),
          'RG012':trial.suggest_int('RG012', 16, 512),
        'RG022':trial.suggest_int('RG022', 16, 512),
        'RG112':trial.suggest_int('RG112', 16, 512),
        'RG122':trial.suggest_int('RG122', 16, 512),
          
          }

    model = TarNet(params)

    pehe,model= train_evaluate(params, model, trial,i)

    return pehe

In [44]:
class Data(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))
        self.len = self.X.shape[0]
       
    def __getitem__(self, index):
        return self.X[index], self.y[index]
   
    def __len__(self):
        return self.len

In [45]:
def get_data(data_type,file_num):

    if(data_type=='train'):
        data=pd.read_csv(f"Dataset/IHDP_a/ihdp_npci_train_{file_num}.csv")
    else:
        data = pd.read_csv(f"Dataset/IHDP_a/ihdp_npci_test_{file_num}.csv")

    x_data=pd.concat([data.iloc[:,0], data.iloc[:, 1:30]], axis = 1)
    #x_data=data.iloc[:, 5:30]
    y_data=data.iloc[:, 1]
    return x_data,y_data

In [46]:
def get_dataloader(x_data,y_data,batch_size):

    x_train_sr=x_data[x_data['treatment']==0]
    y_train_sr=y_data[x_data['treatment']==0]
    x_train_tr=x_data[x_data['treatment']==1]
    y_train_tr=y_data[x_data['treatment']==1]


    train_data_sr = Data(np.array(x_train_sr), np.array(y_train_sr))
    train_dataloader_sr = DataLoader(dataset=train_data_sr, batch_size=batch_size)

    train_data_tr = Data(np.array(x_train_tr), np.array(y_train_tr))
    train_dataloader_tr = DataLoader(dataset=train_data_tr, batch_size=batch_size)


    return train_dataloader_sr, train_dataloader_tr

In [47]:
def regression_loss(concat_true, concat_pred):
    #computes a standard MSE loss for TARNet
    y_true = concat_true[:, 0] #get individual vectors
    t_true = concat_true[:, 1]

    y0_pred = concat_pred[:, 0]
    y1_pred = concat_pred[:, 1]

    #Each head outputs a prediction for both potential outcomes
    #We use t_true as a switch to only calculate the factual loss
    loss0 = torch.sum((1. - t_true) * torch.square(y_true - y0_pred))
    loss1 = torch.sum(t_true * torch.square(y_true - y1_pred))
    #note Shi uses tf.reduce_sum for her losses instead of tf.reduce_mean.
    #They should be equivalent but it's possible that having larger gradients accelerates convergence.
    #You can always try changing it!
    return loss0 + loss1

In [48]:
def cal_pehe(data,y,model):
    #data,y=get_data('test',i)
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    data=data.to_numpy()
    data=torch.from_numpy(data.astype(np.float32)).to(device)



    concat_pred=model(data[:,5:30])
    #dont forget to rescale the outcome before estimation!
    #y0_pred = data['y_scaler'].inverse_transform(concat_pred[:, 0].reshape(-1, 1))
    #y1_pred = data['y_scaler'].inverse_transform(concat_pred[:, 1].reshape(-1, 1))
    cate_pred=concat_pred[:,1]-concat_pred[:,0]
    cate_true=data[:,4]-data[:,3] #Hill's noiseless true values


    cate_err=torch.mean( torch.square( ( (cate_true) - (cate_pred) ) ) )

    return torch.sqrt(cate_err).item()


In [49]:


def loss_cal(X_data,y_data,net,device):
    
    x_train_sr=X_data[X_data['treatment']==0]
    y_train_sr=y_data[X_data['treatment']==0]
    x_train_tr=X_data[X_data['treatment']==1]
    y_train_tr=y_data[X_data['treatment']==1]
    xs_t=x_train_sr.iloc[:,0].to_numpy()
    xt_t=x_train_tr.iloc[:,0].to_numpy()
    
    xs=x_train_sr.iloc[:,5:30].to_numpy()
    xt=x_train_tr.iloc[:,5:30].to_numpy()
    xs_t=torch.from_numpy(xs_t.astype(np.float32))
    xt_t=torch.from_numpy(xt_t.astype(np.float32))
    y_train_sr=y_train_sr.to_numpy()
    y_train_tr=y_train_tr.to_numpy()
    xs=torch.from_numpy(xs.astype(np.float32))
    xt=torch.from_numpy(xt.astype(np.float32))
    
    y_train_sr=torch.from_numpy(y_train_sr.astype(np.float32))
    y_train_tr=torch.from_numpy(y_train_tr.astype(np.float32))
    
    
    input_data=torch.cat((xs,xt),0).to(device)
    true_y=torch.unsqueeze(torch.cat((y_train_sr,y_train_tr),0), dim=1).to(device)
    true_t=torch.unsqueeze(torch.cat((xs_t,xt_t),0), dim=1).to(device)
    
    
    concat_true=torch.cat((true_y,true_t),1)
    concat_pred=net(input_data)
    loss=regression_loss(concat_true, concat_pred)
    loss_2=y_MSE(concat_pred[0],concat_pred[1])
    return loss.item()

def cf_loss(xs,xt):

        #col =  ["treatment", "y_factual", "y_cfactual",]
        #for i in range(1,28):
        #    col.append("x"+str(i))

        #df_datac=pd.DataFrame(xs.numpy(),columns=col)
        #df_datat=pd.DataFrame(xt.numpy(),columns=col)
        
                
        PhiC=xs[:,5:30]
        PhiT=xt[:,5:30]
              
        
        dists = torch.sqrt(torch.cdist(PhiC, PhiT))
        #c_index=torch.argmin(dists, dim=0).tolist()
        #t_index=torch.argmin(dists, dim=1).tolist()
        #yT_nn=df_datac.iloc[c_index]['y_factual']
        #yC_nn=df_datat.iloc[t_index]['y_factual']
        
        
        c_index=torch.argmin(dists, dim=0)
        t_index=torch.argmin(dists, dim=1)
        yT_nn=xs[c_index,1]
        yC_nn=xt[t_index,1]
        
        
        #yT_nn=yT_nn.to_numpy()
        #yT_nn=torch.from_numpy(yT_nn.astype(np.float32))
        #yC_nn=yC_nn.to_numpy()
        #yC_nn=torch.from_numpy(yC_nn.astype(np.float32))
       
        return yC_nn,yT_nn
    
def cf_loss_prop(xs,xt):

        #col =  ["treatment", "y_factual", "y_cfactual",]
        #for i in range(1,28):
        #    col.append("x"+str(i))

        #df_datac=pd.DataFrame(xs.numpy(),columns=col)
        #df_datat=pd.DataFrame(xt.numpy(),columns=col)
        
                
        PhiC=xs[:,5:30]
        PhiT=xt[:,5:30]
              
        
        dists = torch.sqrt(torch.cdist(PhiC, PhiT))
        #c_index=torch.argmin(dists, dim=0).tolist()
        #t_index=torch.argmin(dists, dim=1).tolist()
        #yT_nn=df_datac.iloc[c_index]['y_factual']
        #yC_nn=df_datat.iloc[t_index]['y_factual']
        
        
        c_index=torch.argmin(dists, dim=0)
        t_index=torch.argmin(dists, dim=1)
        yT_nn=xs[c_index,1]
        yC_nn=xt[t_index,1]
        
        
        #yT_nn=yT_nn.to_numpy()
        #yT_nn=torch.from_numpy(yT_nn.astype(np.float32))
        #yC_nn=yC_nn.to_numpy()
        #yC_nn=torch.from_numpy(yC_nn.astype(np.float32))
       
        return yC_nn,yT_nn

In [50]:

y_MSE=nn.MSELoss()
#criterion_reg=regression_loss(concat_true,concat_pred)
epochs=300
#batch_size=32

In [51]:
torch.cuda.is_available()

True

In [52]:
train_loss=[]
val_loss=[]
pehe_error=[]
num_files=2
def train_evaluate(param, model, trial,file_num):
    #for nf in range(1,num_files):
    x_data,y_data=get_data('train',file_num)
    X_train, X_val,y_train, y_val = train_test_split(x_data,y_data ,
                                       random_state=42, 
                                       test_size=0.20)
    
    #net=TarNet(25,.01)
    #opt_net = torch.optim.Adam(net.parameters(), lr=1e-4)
    
   
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    optimizer = getattr(optim, param['optimizer'])(model.parameters(), lr= param['learning_rate'])
    
    #if use_cuda:

        #model = model.cuda()
    model = model.to(device)
        #criterion = criterion.cuda()

    for ep in range(1,epochs+1 ):

        train_dataloader_sr, train_dataloader_tr=get_dataloader(X_train,y_train,param['batch_size'])

        for batch_idx, (train_source_data, train_target_data) in enumerate(zip(train_dataloader_sr, train_dataloader_tr)):

            xs,ys=train_source_data
            xt,yt=train_target_data
            yC_nn,yT_nn=cf_loss(xs,xt)
            #print(xs)

            xs_train=xs[:,5:30]
            xt_train=xt[:,5:30]

            train_x=torch.cat((xs_train,xt_train),0).to(device)
            train_y=torch.unsqueeze(torch.cat((ys,yt),0), dim=1).to(device)
            true_t=torch.unsqueeze(torch.cat((xs[:,0],xt[:,0]),0), dim=1).to(device)
            concat_true=torch.cat((train_y,true_t),1).to(device)
            concat_pred=model(train_x).to(device)

            model.zero_grad()

            #source_mse=criterion_reg(y0,ys)
            #target_mse=criterion_reg(y1,yt)
            #cf_los=(y_MSE(concat_pred[0:xs_train.shape[0],1],yC_nn.to(device)))+(y_MSE(concat_pred[xs_train.shape[0]:,0],yT_nn.to(device)))
            cf_los_1=torch.sum(torch.abs(concat_pred[0:xs_train.shape[0],1]-yC_nn.to(device)))
            cf_los_2=torch.sum(torch.abs(concat_pred[xs_train.shape[0]:,0]-yT_nn.to(device)))
            cf_los=cf_los_1+cf_los_2
            #combined loss
            combined_loss=regression_loss(concat_true,concat_pred)+cf_los #add tradeoff term here with between two
            #losses
                        #print('Training loss: ',combined_loss.item())
            # backward propagation
            combined_loss.backward()

            # optimize
            optimizer.step()
        #train_loss.append(loss_cal(X_train,y_train,net))
        #val_loss.append(loss_cal(X_val,y_val,net))
        
        # Add prune mechanism
        #trial.report(accuracy, ep)

        #if trial.should_prune():
        #   raise optuna.exceptions.TrialPruned()
            
    #return cal_pehe(X_val,y_val,model),model
    return loss_cal(X_val,y_val,model,device),model

        
        

In [53]:
pehe_total=[]
for i in range(1,101):
    func = lambda trial: objective(trial, i)
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(func, n_trials=50)
    best_trial = study.best_trial
    best_model=TarNet(study.best_trial.params)
    best_val,model=train_evaluate(study.best_trial.params, best_model, study.best_trial,i)
    data,y=get_data('test',i)
    pehe=cal_pehe(data,y,model)

    pehe_total.append(pehe)


[I 2023-05-18 13:36:57,050] A new study created in memory with name: no-name-2c11362e-245b-4d6c-8592-ba88180b9c15
/tmp/ipykernel_10110/1969858443.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-3),
[I 2023-05-18 13:36:59,701] Trial 0 finished with value: 435.701416015625 and parameters: {'learning_rate': 5.6115164153345e-05, 'optimizer': 'Adam', 'batch_size': 157, 'RL11': 93, 'RL21': 93, 'RL32': 44, 'RG012': 446, 'RG022': 314, 'RG112': 367, 'RG122': 26}. Best is trial 0 with value: 435.701416015625.
/tmp/ipykernel_10110/1969858443.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` in

[I 2023-05-18 13:37:35,760] Trial 12 finished with value: 227.366455078125 and parameters: {'learning_rate': 0.00014595432633183366, 'optimizer': 'SGD', 'batch_size': 255, 'RL11': 191, 'RL21': 504, 'RL32': 390, 'RG012': 115, 'RG022': 20, 'RG112': 37, 'RG122': 179}. Best is trial 11 with value: 205.8536376953125.
/tmp/ipykernel_10110/1969858443.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-3),
[I 2023-05-18 13:37:38,168] Trial 13 finished with value: 208.11085510253906 and parameters: {'learning_rate': 0.00013450549446761032, 'optimizer': 'SGD', 'batch_size': 170, 'RL11': 191, 'RL21': 408, 'RL32': 371, 'RG012': 22, 'RG022': 19, 'RG112': 215, 'RG122': 177}. Best is trial 11 with value: 205.8536376953125.
/tmp/ipykernel_10110/1969858443.py:4: Fu

[W 2023-05-18 13:38:04,286] Trial 24 failed with parameters: {'learning_rate': 0.00022165082237284606, 'optimizer': 'SGD', 'batch_size': 162, 'RL11': 18, 'RL21': 383, 'RL32': 474, 'RG012': 342, 'RG022': 87, 'RG112': 225, 'RG122': 188} because of the following error: The value nan is not acceptable..
[W 2023-05-18 13:38:04,287] Trial 24 failed with value nan.
/tmp/ipykernel_10110/1969858443.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-3),
[W 2023-05-18 13:38:06,356] Trial 25 failed with parameters: {'learning_rate': 0.00018284671996493518, 'optimizer': 'SGD', 'batch_size': 161, 'RL11': 500, 'RL21': 376, 'RL32': 494, 'RG012': 125, 'RG022': 93, 'RG112': 213, 'RG122': 188} because of the following error: The value nan is not acceptable..
[W 2023

[W 2023-05-18 13:38:30,094] Trial 35 failed with value nan.
/tmp/ipykernel_10110/1969858443.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-3),
[W 2023-05-18 13:38:32,520] Trial 36 failed with parameters: {'learning_rate': 0.00019293189794755531, 'optimizer': 'SGD', 'batch_size': 160, 'RL11': 21, 'RL21': 380, 'RL32': 475, 'RG012': 113, 'RG022': 94, 'RG112': 235, 'RG122': 176} because of the following error: The value nan is not acceptable..
[W 2023-05-18 13:38:32,521] Trial 36 failed with value nan.
/tmp/ipykernel_10110/1969858443.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` in

[W 2023-05-18 13:38:58,683] Trial 47 failed with parameters: {'learning_rate': 0.00020549770560630956, 'optimizer': 'SGD', 'batch_size': 166, 'RL11': 207, 'RL21': 378, 'RL32': 486, 'RG012': 129, 'RG022': 88, 'RG112': 201, 'RG122': 188} because of the following error: The value nan is not acceptable..
[W 2023-05-18 13:38:58,684] Trial 47 failed with value nan.
/tmp/ipykernel_10110/1969858443.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-3),
[W 2023-05-18 13:39:01,068] Trial 48 failed with parameters: {'learning_rate': 0.000192119671471052, 'optimizer': 'SGD', 'batch_size': 165, 'RL11': 504, 'RL21': 383, 'RL32': 472, 'RG012': 333, 'RG022': 98, 'RG112': 217, 'RG122': 183} because of the following error: The value nan is not acceptable..
[W 2023-

In [54]:
print(np.mean(pehe_total))

0.9562848210334778


In [176]:
pehe_total

[0.7275428175926208, 0.9615147113800049]

In [14]:
#np.mean(pehe_total[0:99])

In [15]:
pehe_total

[0.7245088219642639,
 1.0319709777832031,
 0.8922997713088989,
 0.766045093536377,
 1.176272988319397,
 1.3779881000518799,
 0.33619603514671326,
 0.9627038240432739,
 0.8635401725769043,
 1.3651361465454102,
 0.9023053050041199,
 0.7707643508911133,
 0.6479784250259399,
 1.3890775442123413,
 1.2930279970169067,
 0.8717934489250183,
 0.5604594945907593,
 0.4141177237033844,
 0.9313862323760986,
 1.3393807411193848,
 0.9354116320610046,
 0.5097612142562866,
 0.8281165957450867,
 0.6784378290176392,
 0.9889419674873352,
 0.8551837205886841,
 0.976718544960022,
 0.8554731607437134,
 1.5037286281585693,
 0.6007119417190552,
 0.5339930653572083,
 0.6747851967811584,
 0.815698504447937,
 0.8607384562492371,
 0.7166107892990112,
 0.6790837049484253,
 0.567247748374939,
 0.45495477318763733,
 1.0859042406082153,
 1.9381632804870605,
 1.7002204656600952,
 0.7606197595596313,
 0.34149885177612305,
 0.8747959733009338,
 0.5832961797714233,
 0.5637949705123901,
 0.8216122984886169,
 1.402480363845

In [16]:
np.savetxt("V2_CFR_y_loss_1_100_(IHDPa-Hyper_val_300ep_outsample).csv", pehe_total,delimiter =", ", fmt ='% s')

In [44]:
#for key, value in best_trial.params.items():
#    print("{}: {}".format(key, value))

[]

In [ ]:
#ate_pred=torch.mean(cate_pred)
#print("Estimated ATE (True is 4):", ate_pred.detach().numpy(),'\n\n')

#print("Individualized CATE Estimates: BLUE")
#print(pd.Series(cate_pred.detach().numpy()).plot.kde(color='blue'))
#print("Individualized CATE True: Green")
#print(pd.Series(cate_true.detach().numpy()).plot.kde(color='green'))

#print("\nError CATE Estimates: RED")
#print(pd.Series(cate_pred.detach().numpy()-cate_true.detach().numpy()).plot.kde(color='red'))